In [13]:
import pandas as pd
from sklearn import preprocessing
from sklearn import svm

def serialize(dataFrame, column):
    return [x for x in range(len(dataFrame[column].unique()))]

def replaceDict(dataFrame, column):
    vals = serialize(dataFrame, column)
    return dict(zip(dataFrame[column].unique(), vals))

UNREST_COLUMNS = ["EVENT_ID_CNTY", 
                    "EVENT_DATE", 
                    "EVENT_TYPE", 
                    "REGION", 
                "FATALITIES"]

CASES_COLUMNS = ["iso_code",
                "continent",
                "location", 
                "date", 
                "total_cases", 
                "new_cases", 
                "total_deaths", 
                "reproduction_rate", 
                "hosp_patients", 
                "positive_rate", 
                "stringency_index", 
                "population",
                "median_age",
                "gdp_per_capita",
                "life_expectancy"
            ]

unrest_df = pd.read_csv("./coronavirus_Oct31.csv")
covid_cases_df = pd.read_csv("./owid-covid-data.csv")

unrest_df = unrest_df[unrest_df.columns.intersection(UNREST_COLUMNS)]
covid_cases_df = covid_cases_df[covid_cases_df.columns.intersection(CASES_COLUMNS)]


unrest_afg = unrest_df[unrest_df["EVENT_ID_CNTY"].str.contains("AFG")]
cases_afg = covid_cases_df.query("iso_code == 'AFG'")

unrest_afg.EVENT_DATE = pd.to_datetime(unrest_afg.EVENT_DATE)
cases_afg.date = pd.to_datetime(cases_afg.date)

merge = unrest_afg.merge(cases_afg, how="inner", left_on="EVENT_DATE", right_on="date")

classification = serialize(unrest_df, "EVENT_TYPE")

issueType = unrest_afg['EVENT_TYPE']
issueType = issueType.replace(replaceDict(unrest_df, "EVENT_TYPE"))

# print(issueType)

# print(classification)

# print(replaceDict(unrest_df, "EVENT_TYPE"))

merge = merge.drop(['EVENT_TYPE', 'EVENT_DATE', 'EVENT_ID_CNTY', 'REGION', 'iso_code', 'continent', 'location', 'date'], axis=1).fillna(0)

# mergeNormal = normalize(merge)
min_max_scaler = preprocessing.MinMaxScaler()

mergeNormal = min_max_scaler.fit_transform(merge)
mergeNormal = pd.DataFrame(mergeNormal)
mergeNormal

clf = svm.SVC()
clf.fit(mergeNormal, issueType)
print(issueType)
clf.predict(mergeNormal)

# issueType





24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    2
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    1
45    0
46    1
47    1
48    0
49    0
50    1
51    0
52    2
53    1
54    1
55    1
56    3
Name: EVENT_TYPE, dtype: int64


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3])